In [1]:
OpenAI_API = "sk-proj-Rl41N05dYzhdjkHneWI2T3BlbkFJXf1SmQ7Nth4wHBCgzHjO"

In [7]:
import openai

In [8]:
client = openai.AsyncOpenAI(api_key=OpenAI_API)

In [6]:
import faiss
import json
import numpy as np
from docx import Document
from openai import OpenAI
from autogen import AssistantAgent

client = OpenAI(api_key=OpenAI_API)

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    return "\n".join([para.text.strip() for para in doc.paragraphs if para.text.strip()])

# 解析 .docx 和 .json 数据
docx_text = extract_text_from_docx("AI Doc.docx")

with open("jetbay-intro.json", "r", encoding="utf-8") as f:
    faq_data = json.load(f)

# 构建知识库数据
data = [{"text": docx_text, "source": "docx"}] + [{"text": item["question"] + " " + item["answer"], "source": "json"} for item in faq_data]

# 计算文本嵌入
def get_embedding(text):
    response = client.embeddings.create(input=[text], model="text-embedding-ada-002")  # Wrap input in a list
    return np.array(response.data[0].embedding) 

embeddings = np.array([get_embedding(item["text"]) for item in data])

# 创建 FAISS 索引
dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings)

# 存储索引
faiss.write_index(faiss_index, "jetbay_faiss.index")

In [4]:
# **定义 FAISS 检索代理**
class FAISSRetriever:
    def __init__(self, index_path, data):
        self.index = faiss.read_index(index_path)
        self.data = data
    
    def search(self, query, top_k=2):
        query_embedding = get_embedding(query).reshape(1, -1)
        D, I = self.index.search(query_embedding, top_k)
        results = [self.data[i]["text"] for i in I[0]]
        return results

retriever = FAISSRetriever("jetbay_faiss.index", data)


agent_prompt = f"""
You are an intelligent assistant in JetBay tasked with answering user queries based on the provided context.  
Use the context information to generate precise, relevant, and natural responses.  
Do not mention the existence of the context or that you are referencing it.  
"""


# **创建 LLM 代理**
llm_agent = AssistantAgent(
    system_message=agent_prompt,
    name="llm",
    model_client_stream=True,
    llm_config={
        "model": "gpt-4o",
        "api_key": OpenAI_API,
        "temperature": 0.7
    }
)

TypeError: ConversableAgent.__init__() got an unexpected keyword argument 'model_client_stream'

In [80]:
chat_history = []

# **执行查询**
# query = "Where are JETBAY's offices located?"
# query = "Does JETBAY offer international flights?"
# query = "What are the advantages of JetBay?"
# query = "Why do I want to choose Jetbay?"
query = "When was JetBay founded and how much profits does it generate?"
# query = "who are you"
# query = "what is jetbay?"
# query = "How do I register in jetbay?"
# query = "Can you help me arrange ground transportation?"
retrieved_context = retriever.search(query, top_k=3)
print(retrieved_context)
chat_history.append({"role": "user", "content": query})
conversation = chat_history + [{"role": "system", "content": f"Context: {retrieved_context}"}]
# **让 LLM 代理生成答案**
# final_prompt = f"""
# ### Context:
# {retrieved_context}

# ### User Query:
# {query}

# ### Response:
# """
# final_prompt = f"Context: {retrieved_context}\n\nUser: {query}\n\nTry to chat or answer the questions given the context, DO NOT MENTION you have context info to reference."
# llm_response = llm_agent.generate_reply([{"role": "user", "content": final_prompt}])  # FIXED
llm_response = llm_agent.generate_reply(conversation)
chat_history.append({"role": "assistant", "content": llm_response})
print(llm_response)

["Founder Statement\nJETBAY is at the forefront of integrating AI and Big Data to offer a more sustainable and intelligent flying experience. We pride ourselves on delivering solutions that are not only the fastest and most comprehensive but also cost-efficient, ensuring our customers always receive the best service possible.\nIntroduction\nFounded in 2020 with headquarters in Singapore and Shanghai, JETBAY has quickly expanded, establishing branch offices in Hong Kong, Beijing and Jakarta. By2026, we aim to strengthen our presence across East and Southeast Asia, with plans to set up offices in major global cities by 2030.\nOur team of over 100 professionals brings unmatched expertise, with the management team averaging over20 years of industry experience.\nBetween 2020 and 2023, our total revenue skyrocketed from USD 43 million to USD 150 million, reflecting an impressive average annual growth rate of 155%.\nJETBAY specialises in crafting high-end charter experiences tailored to each 

In [81]:
chat_history

[{'role': 'user',
  'content': 'When was JetBay founded and how much profits does it generate?'},
 {'role': 'assistant',
  'content': 'JetBay was founded in 2020. While specific profit figures are not provided, the company experienced significant revenue growth from USD 43 million in 2020 to USD 150 million in 2023, indicating substantial financial success and expansion.'}]